In [3]:
import os
import torch
import json
from io import BytesIO
import numpy as np
import torch.nn as nn
import imageio.v2 as imageio
from tqdm import tqdm, trange
from utils import SphericalNODataset
from neuralop.models import SFNO
import matplotlib.pyplot as plt

/Users/reza/Career/DMLab/SURROGATE/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
device = torch.device('cpu')

In [5]:
data_path = '/Users/reza/Career/DMLab/SURROGATE/Data/psi_web_sample/test'
result_path = '/Users/reza/Career/DMLab/SURROGATE/results/laplace/sfno/exp_12'
cfg_path = os.path.join(result_path, 'cfg.json')
with open(cfg_path, 'r') as f:
    cfg = json.load(f)

In [6]:
len(cfg['train_files'])

680

In [7]:
instruments = [
        "kpo_mas_mas_std_0101",
        "mdi_mas_mas_std_0101",
        "hmi_mast_mas_std_0101",
        "hmi_mast_mas_std_0201",
        "hmi_masp_mas_std_0201",
        "mdi_mas_mas_std_0201",
    ]
subdir_paths = sorted(os.listdir(data_path))
cr_paths = [os.path.join(data_path, p) for p in subdir_paths if p.startswith("cr")]
sim_paths = []
for cr_path in cr_paths:
    for instrument in instruments:
        instrument_path = os.path.join(cr_path, instrument)
        if os.path.exists(instrument_path):
            sim_paths.append(instrument_path)

In [8]:
dataset = SphericalNODataset(
    sim_paths,
    b_min=cfg["train_min"],
    b_max=cfg["train_max"],
)

100%|██████████| 21/21 [00:00<00:00, 50.23it/s]


In [9]:
cfg['train_min'], cfg['train_max']

(-2.6867804527282715, 2.7947838306427)

In [11]:
model = SFNO(
    n_modes=(32, 32),
    in_channels=1,
    out_channels=140,
    hidden_channels=cfg['hidden_channels'],
    projection_channel_ratio=2,
    factorization="dense",
)
state = torch.load(os.path.join(result_path, 'ckpt', "best_model_state_dict.pt"), map_location=device, weights_only=False)
model.load_state_dict(state)

<All keys matched successfully>

In [16]:
instance = dataset[0]

In [48]:
I = 10
instance = dataset[I]
x, y = instance['x'], instance['y']

model.eval()
with torch.no_grad():
    yhats = model(x.unsqueeze(0))

In [49]:
y.shape, yhats.shape

(torch.Size([140, 110, 128]), torch.Size([1, 140, 110, 128]))

In [50]:
cube = y

In [51]:
frames = []
for i in trange(140):
    y = cube[i, :, :]
    yhat = yhats[0, i, :, :]
    # print(cube.shape)
    # y = y.reshape((128, 110))
    # # print(yhat.shape)
    # yhat = yhat.reshape((128, 110))
    # y = y.transpose(1, 0)
    # yhat = yhat.transpose(1, 0)

    error = np.abs(y - yhat)
    vmin = min(y.min(), yhat.min())
    vmax = max(y.max(), yhat.max())
    fig, axes = plt.subplots(1, 3, figsize=(12, 6))
    cmap = "coolwarm"

    # # Plot first subplot
    im1 = axes[0].imshow(y, cmap=cmap, vmin=vmin, vmax=vmax)
    axes[0].set_title(f"gt: br002 at {i+1}")

    # Plot second subplot
    im3 = axes[1].imshow(yhat, cmap=cmap, vmin=vmin, vmax=vmax)
    axes[1].set_title(f"pred at {i+1} ")

    cbar = fig.colorbar(im1, ax=axes, orientation="horizontal", fraction=0.1, pad=0.02)

    # Plot second subplot
    im4 = axes[2].imshow(error, cmap="gray")
    axes[2].set_title(f"|pred-gt|")

    # Create a single colorbar
    cbar = fig.colorbar(im4, ax=axes, orientation="vertical", fraction=0.05, pad=0.02)

    # plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format="png")
    frames.append(imageio.imread(buf))
    plt.close()

100%|██████████| 140/140 [00:24<00:00,  5.60it/s]


In [52]:
output_filename = f"b{I}.gif"
fps = 10  # Adjust frames per second as needed

# Create video directly from frames in memory
with imageio.get_writer(output_filename, fps=fps, loop=0) as writer:
    for frame in frames:
        writer.append_data(frame)